### **Approach 1: Creating and populating the database with SQLite**

Since SQLite is server-less and self-contained I figured this would be a good starting point for a prototype.\
I created some scripts with sqlite3 to create and populate the database with authors, books, stores, customers, orders etc.

I ended up with these tables:

<div style="display: flex; flex-direction: row;">
  <div style="flex: 1; margin:5px;">

**Table 1: Author**
- 'ID' (Primary Key)
- 'Name'
- 'Surname'
- 'Birthdate'

**Table 2: Book**
- 'ISBN13' (Primary Key)
- 'Title'
- 'Language'
- 'Price'
- 'Release'
- 'AuthID'


**Table 3: Store**
- 'ID' (Primary Key)
- 'Store_Name'
- 'Store_Address'

</div>
<div style="flex: 1; margin:5px;">

**Table 4: Inventory**
- 'StoreID'
- 'ISBN13'
- 'Stock'
- Composite Key: 'StoreID' & 'ISBN13'

**Table 5: Cst (Customer)**
- 'ID' (Primary Key)
- 'Name'
- 'Surname'
- 'Address'
- 'City'
- 'State'
- 'ZipCode'
- 'Email'

</div>
<div style="flex: 1; margin:5px;">

**Table 6: Cst_Order**
- 'OrderID' (Primary Key)
- 'CustomerID'
- 'Order_Date'
- 'Expected_Delivery'
- 'Total_Price'

**Table 7: Cst_Order_Items**
- 'OrderID (Primary Key)
- 'ISBN13' (Primary key)
- 'Units'
- 'Shipped_From'
- Composite Key: 'OrderID' & 'ISBN13'
</div>

The requriement of the view *Vy: ”TitlarPerFörfattare”*, was created with some joins:

In [ ]:
c.execute('''CREATE VIEW TitlarPerFörfattare AS
               SELECT Author.Name || ' ' || Author.Surname AS Namn,
                      strftime('%Y', 'now') - strftime('%Y', Author.Birthdate) AS Ålder,
                      COUNT(DISTINCT Book.Title) AS Titlar,
                      SUM(Book.Price * Inventory.Stock) AS Lagervärde
               FROM Author
               JOIN Book ON Author.ID = Book.AuthID
               JOIN Inventory ON Book.ISBN13 = Inventory.ISBN13
               GROUP BY Author.Name, Author.Surname
               ORDER BY Namn''')

**Sample data**, such as customer info, was easy to generate with chatGPT.\
I made a function called populate_customer_table(), in which I added these rows:

In [ ]:
    customer_data = [
        ('John', 'Doe', 'johndoe@example.com', '123 Main St', 'New York', 'NY', '10001'),
        ('Jane', 'Smith', 'janesmith@example.com', '456 Elm St', 'Los Angeles', 'CA', '90001'),
        ('Bob', 'Johnson', 'bobjohnson@example.com', '789 Oak St', 'Chicago', 'IL', '60601'),
        ('Alice', 'Lee', 'alicelee@example.com', '321 Maple St', 'Houston', 'TX', '77001'),
        ('David', 'Brown', 'davidbrown@example.com', '654 Pine St', 'Philadelphia', 'PA', '19101')
    ]

But I soon realized I'd actually already made a program for this, <a href="https://github.com/wlinds/tpme">tpme</a>.

tpme can generate names and email addresses but **not** physical home addresses.

So should I focus on adding support for home address generation in tpme or should I port the SQLite code to a larger dbms?

Neither -- *I have like 120 hours left until deadline. No problem. I'll work on some front end stuff.*

#### **Front-end hours**

I started working on an interface with Flask to allow users to purchase books. But then it struck me that this was a total waste of time, I should rather make an interface for the shopkeepers at the store. Then we just outsource the customer-site creation and just hand them some API.

---

**About ISBN**

Example of how an ISBN is structured: ISBN 978-91-7000-150-5

1. Prefix - all ISBNs begin with the prefix 978
2. Area identifier - for nationally, geographically or linguistically determined areas (91 = Sweden)
3. Publisher identifier (7000)
4. Title identifier (150)
5. Check digit (5)

<a href="https://www.kb.se/download/18.280a529018702e6727e363d/1681904094677/ISBN-2023.pdf">source</a>

This might cause search issues. Users might query a seach as "978-91-7000-150-5", "978-9170001505" or "9789170001505".

To solve this, we always strip search queries: 

In [3]:
from utils import validate_isbn

# Validate ISBN
# Source https://rosettacode.org/wiki/ISBN13_check_digit

# Validate the check digit of an ISBN-13 code:

#  Multiply every other digit by  3.
#  Add these numbers and the other digits.
#  Take the remainder of this number after division by  10.
#  If it is  0,   the ISBN-13 check digit is correct.

tests = '''
978-91-7000-150-5
978-1734314509
978-1788399081
978-1788399083'''.strip().split()
for t in tests:
    print(f"ISBN13 {t} validates {validate_isbn(t)}")

ISBN13 978-91-7000-150-5 validates True
ISBN13 978-1734314509 validates False
ISBN13 978-1788399081 validates True
ISBN13 978-1788399083 validates False


Also, ISBN should be stored as text data and not int, since some ISBN can start with 0.